In [2]:
from os import listdir, path as os_path

import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Activation
from tensorflow.keras import optimizers
from tensorflow.nn import relu, softmax

In [2]:
data_path = 'data/AstroDataset'
allowed_classes = {'C', 'M', 'Mrk', 'sd', 'PN'}

X = []
y = []

for c in listdir(data_path):
    c_path = os_path.join(data_path, c)
    for im_name in listdir(c_path):
        im_path = os_path.join(c_path, im_name)
        im = cv2.imread(im_path, cv2.IMREAD_GRAYSCALE)
        if im.shape[0] != 140 or im.shape[1] != 20:
            im = cv2.resize(im, (20, 140))
        if c in allowed_classes:
            X.append(im)
            y.append(c)

X = np.array(X)

In [4]:
le = LabelEncoder()
print(np.unique(y, return_counts=True)[0])
y = le.fit_transform(y)
print(np.unique(y, return_counts=True))

['C' 'M' 'Mrk' 'PN' 'sd']
(array([0, 1, 2, 3, 4]), array([425, 198, 391,  15, 707]))


In [5]:
y = to_categorical(y, len(allowed_classes))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.2)

In [7]:
X_train.shape, y_test.shape

((1388, 140, 20), (348, 5))

In [8]:
input_shape = (140, 20, 1)
X_train = X_train.reshape(X_train.shape[0], input_shape[0], input_shape[1], input_shape[2])
X_test = X_test.reshape(X_test.shape[0], input_shape[0], input_shape[1], input_shape[2])

In [9]:
X_train.shape

(1388, 140, 20, 1)

In [10]:
model_sm = Sequential()
model_sm.add(Conv2D(128, kernel_size=(3,3), input_shape=input_shape, padding="same"))
model_sm.add(MaxPooling2D(pool_size=(3, 1)))

model_sm.add(Conv2D(64, (3, 3), padding="same"))
model_sm.add(Activation('relu'))
model_sm.add(MaxPooling2D(pool_size=(3, 1)))

model_sm.add(Conv2D(32, (3, 3), padding="same"))
model_sm.add(Activation('relu'))
model_sm.add(MaxPooling2D(pool_size=(3, 4)))

model_sm.add(Flatten()) # Flattening the 2D arrays for fully connected layers
model_sm.add(Dense(128, activation=relu))
model_sm.add(Dropout(0.5))
model_sm.add(Dense(len(allowed_classes), activation=softmax))

2022-08-06 17:54:41.754929: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-06 17:54:41.754986: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mlgod-GF63-Thin-11UC): /proc/driver/nvidia/version does not exist
2022-08-06 17:54:41.755371: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
#image data generator keras
adadelta = optimizers.Adadelta(learning_rate=0.1, rho=0.95)
model_sm.compile(optimizer=adadelta , loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
history_sm = model_sm.fit(X_train, y_train, epochs = 100, batch_size = 64, shuffle=True, validation_data=(X_test, y_test))

Epoch 1/100


2022-08-06 17:54:42.225358: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 91750400 exceeds 10% of free system memory.
2022-08-06 17:54:42.298470: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30146560 exceeds 10% of free system memory.
2022-08-06 17:54:42.355869: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 27648000 exceeds 10% of free system memory.
2022-08-06 17:54:42.355910: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 27648000 exceeds 10% of free system memory.
2022-08-06 17:54:42.393726: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 30146560 exceeds 10% of free system memory.


22/22 [==============================] - 7s 301ms/step - loss: 1.4530 - accuracy: 0.3228 - val_loss: 1.1218 - val_accuracy: 0.4310
Epoch 2/100
21/22 [===========================>..] - ETA: 0s - loss: 0.5705 - accuracy: 0.4040

KeyboardInterrupt: 

In [ ]:
plt.plot(history_sm.history['accuracy'], label='Accuracy (train data)')
plt.plot(history_sm.history['val_accuracy'], label='Accuracy (val data)')
plt.title('Accuracy on DFBS Images')
plt.ylabel('Accuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
plt.plot(history_sm.history['loss'], label='MSE (train data)')
plt.plot(history_sm.history['val_loss'], label='MSE (val data)')
plt.title('MSE for DFBS Images')
plt.ylabel('MSE value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
y_pred = model_sm.predict(X_test)
print(classification_report(y_test, to_categorical(np.argmax(y_pred, axis=1))))